<div class="alert alert-block alert-info">
<b>Number of points for this notebook:</b> 1
<br>
<b>Deadline:</b> March 23, 2020 (Monday). 23:00
</div>

# Exercise 4.2. Convolutional networks. VGG-style network.

In the second part you need to train a convolutional neural network with an architecture inspired by a VGG-network [(Simonyan \& Zisserman, 2015)](https://arxiv.org/abs/1409.1556).

In [1]:
skip_training = True  # Set this flag to True before validation and submission

In [2]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tools
import tests

In [5]:
# When running on your own computer, you can specify the data directory by:
# data_dir = tools.select_data_dir('/your/local/data/directory')
data_dir = tools.select_data_dir()

The data directory is ../data


In [6]:
# Select the device for training (use GPU if you have one)
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [7]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## FashionMNIST dataset

Let us use the FashionMNIST dataset. It consists of 60,000 training images of 10 classes: 'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'.

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize((0.5,), (0.5,))  # Min-max scaling to [-1, 1]
])

trainset = torchvision.datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root=data_dir, train=False, download=True, transform=transform)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
           'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

93.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# VGG-style network

Let us now define a convolution neural network with an architecture inspired by the [VGG-net](https://arxiv.org/abs/1409.1556):

<img src="vgg-style.png" width=600 style="float: left;">

The architecture:
* A block of three convolutional layers with:
    * 3x3 kernel
    * 16 output channels
    * one pixel zero-pading on both sides
    * 2d batch normalization after each convolutional layer
    * ReLU nonlinearity after each 2d batch normalization layer
* Max pooling layer with 2x2 kernel and stride 2.
* A block of three convolutional layers with:
    * 3x3 kernel
    * 32 output channels
    * one pixel zero-pading on both sides
    * 2d batch normalization after each convolutional layer
    * ReLU nonlinearity after each 2d batch normalization layer
* Max pooling layer with 2x2 kernel and stride 2.
* One convolutional layer with:
    * 3x3 kernel
    * 48 output channels
    * *no padding*
    * 2d batch normalization after the convolutional layer
    * ReLU nonlinearity after the 2d batch normalization layer
* One convolutional layer with:
    * 1x1 kernel
    * 32 output channels
    * *no padding*
    * 2d batch normalization after the convolutional layer
    * ReLU nonlinearity after the 2d batch normalization layer
* One convolutional layer with:
    * 1x1 kernel
    * 16 output channels
    * *no padding*
    * 2d batch normalization after the convolutional layer
    * ReLU nonlinearity after the 2d batch normalization layer
* Global average pooling (compute the average value of each channel across all the input locations):
    * 5x5 kernel (the input of the layer should be 5x5)
* A fully-connected layer with 10 outputs (no nonlinearity)

Notes:
* Batch normalization is expected to be right after a convolutional layer, before nonlinearity.
* We recommend that you check the number of modules with trainable parameters in your network.

In [21]:
class VGGNet(nn.Module):
    def __init__(self, n_channels=16):
        """
        Args:
          n_channels (int): Number of channels in the first convolutional layer. The number of channels in the following layers are the multipliers of n_channels. Hence, parameters of the layers to follow can be defined using this variable.
        """
        super(VGGNet, self).__init__()
        # YOUR CODE HERE
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 48, 3),
            nn.ReLU(),
            nn.BatchNorm2d(48),
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(48, 32, 1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(32, 16, 1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
        )
        self.layer6 = nn.Sequential(
            nn.AvgPool2d(5)
        )
        self.layer7 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16, 10)
        )

    def forward(self, x, verbose=False):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
          verbose: True if you want to print the shapes of the intermediate variables.
        
        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        # YOUR CODE HERE
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        return x

In [22]:
def test_VGGNet_shapes():
    net = VGGNet()
    net.to(device)

    # Feed a batch of images from the training data to test the network
    with torch.no_grad():
        images, labels = iter(trainloader).next()
        images = images.to(device)
        print('Shape of the input tensor:', images.shape)

        y = net(images, verbose=True)
        assert y.shape == torch.Size([trainloader.batch_size, 10]), f"Bad y.shape: {y.shape}"

    print('Success')

test_VGGNet_shapes()

Shape of the input tensor: torch.Size([32, 1, 28, 28])
Success


In [23]:
tests.test_vgg_net(VGGNet)

y: tensor([[ 8.0032,  8.0032,  8.0032,  8.0032,  8.0032, -8.0032, -8.0032, -8.0032,
         -8.0032, -8.0032]], grad_fn=<AddmmBackward>)
expected: tensor([ 8.0032,  8.0032,  8.0032,  8.0032,  8.0032, -8.0032, -8.0032, -8.0032,
        -8.0032, -8.0032])
Success


# Train the network

In [24]:
# This function computes the accuracy on the test dataset
def compute_accuracy(net, testloader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

### Training loop

Your task is to implement the training loop. The recommended hyperparameters:
* Adam optimizer with learning rate 0.01.
* Cross-entropy loss. Note that we did not use softmax nonlinearity in the final layer of our network. Therefore, we need to use a loss function with log_softmax implemented, such as [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss).
* Number of epochs: 10

We recommend you to use function `compute_accuracy()` defined above to track the accaracy during training. The test accuracy should be above 0.87.

**Note: function `compute_accuracy()` sets the network into the evaluation mode which changes the way the batch statistics are computed in batch normalization. You need to set the network into the training mode (by calling `net.train()`) when you want to perform training.**

In [25]:
net = VGGNet()

In [28]:
# Implement the training loop in this cell
if not skip_training:
    # YOUR CODE HERE
    accu = 0
    for i in range(10):
        if accu > 0.87:
            break
        criterion = nn.CrossEntropyLoss().cuda()
        optimizer = optim.Adam(net.parameters(), lr=0.01)
        for i_batch, sample_batched in enumerate(trainloader):
            if accu > 0.87:
                break
            inputs, labels = sample_batched
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            if i_batch % 100 == 0:  
                accu = compute_accuracy(net, testloader)
                print("Accuracy: %f" % accu)        

epoch: 0
Accuracy: 0.760900
Accuracy: 0.794400
Accuracy: 0.759300
Accuracy: 0.812500
Accuracy: 0.814000
Accuracy: 0.806900
Accuracy: 0.804400
Accuracy: 0.801900
Accuracy: 0.819200
Accuracy: 0.794300
Accuracy: 0.798000
Accuracy: 0.815500
Accuracy: 0.810400
Accuracy: 0.818800
Accuracy: 0.815800
Accuracy: 0.810600
Accuracy: 0.816500
Accuracy: 0.798100
Accuracy: 0.811200
epoch: 1
Accuracy: 0.809400
Accuracy: 0.825700
Accuracy: 0.827700
Accuracy: 0.814400
Accuracy: 0.826100
Accuracy: 0.819900
Accuracy: 0.824600
Accuracy: 0.823400
Accuracy: 0.830300
Accuracy: 0.828400
Accuracy: 0.828400
Accuracy: 0.834200
Accuracy: 0.822500
Accuracy: 0.825300
Accuracy: 0.835900
Accuracy: 0.830400
Accuracy: 0.829400
Accuracy: 0.829000
Accuracy: 0.840100
epoch: 2
Accuracy: 0.843500
Accuracy: 0.827900
Accuracy: 0.823900
Accuracy: 0.844900
Accuracy: 0.828000
Accuracy: 0.837800
Accuracy: 0.844000
Accuracy: 0.842500
Accuracy: 0.836200
Accuracy: 0.833500
Accuracy: 0.833400
Accuracy: 0.830100
Accuracy: 0.842400
Accu

KeyboardInterrupt: 

In [29]:
# Save the model to disk (the pth-files will be submitted automatically together with your notebook)
if not skip_training:
    tools.save_model(net, '4_vgg_net.pth')
else:
    net = VGGNet()
    tools.load_model(net, '4_vgg_net.pth', device)

Do you want to save the model (type yes to confirm)? yes
Model saved to 4_vgg_net.pth.


In [30]:
# Compute the accuracy on the test set
accuracy = compute_accuracy(net, testloader)
print('Accuracy of the VGG net on the test images: %.3f' % accuracy)
assert accuracy > 0.87, "Poor accuracy ({:.3f})".format(accuracy)
print('Success')

Accuracy of the VGG net on the test images: 0.878
Success
